# Query datalakes on S3 with Amazon Athena

Visit the followint AWS blogpost for more detailed instructions: https://aws.amazon.com/blogs/machine-learning/run-sql-queries-from-your-sagemaker-notebooks-using-amazon-athena/

### Install dependencies.
PyAthena is a Python DB API 2.0 (PEP 249) compliant client for the Amazon Athena JDBC driver.

In [5]:
import sys
!{sys.executable} -m pip install PyAthena

### Connect to Athena and run SQL queries against your datalake on S3.

In [15]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://athena-staging-fulu',
               region_name='eu-west-1')

df = pd.read_sql("SELECT * FROM udacity.accidents limit 10;", conn)
df

,accident_id,datetime,severity,distance,description,temperature,wind_speed,precipitation,airport_code,city_id,weather_condition_id,year,month
0,A-581709,2019-02-06,2,0.0,Accident on Madison Rd at Hackberry St.,51.1,5.8,0.16,KLUK,0,25769805053,2019,2
1,A-583037,2019-02-06,3,0.0,Accident on I-40 Eastbound at Exit 192 Mccrory...,59.0,8.1,0.45,KBNA,1434519076865,25769805053,2019,2
2,A-585669,2019-02-07,2,0.0,Accident on Gaston Ave at Munger Blvd.,61.0,9.2,0.05,KDAL,592705486849,25769805053,2019,2
3,A-585670,2019-02-07,3,0.0,Accident on I-30 Westbound at Dolphin Rd.,61.0,9.2,0.05,KDAL,592705486849,25769805053,2019,2
4,A-585673,2019-02-07,3,0.0,Accident on I-30 Westbound before US-75.,61.0,9.2,0.05,KDAL,592705486849,25769805053,2019,2
5,A-585674,2019-02-07,3,0.0,Accident on I-635 Westbound at Miller Rd.,61.0,9.2,0.05,KDAL,592705486849,25769805053,2019,2
6,A-585675,2019-02-07,2,0.0,Accident on Royal Ln at Dennis Rd.,61.0,9.2,0.05,KDAL,592705486849,25769805053,2019,2
7,A-1990679,2019-02-15,2,0.5429999999999999,At Hood Franklin Rd/Exit 504 - Accident. Lane ...,50.0,10.4,0.0,KSAC,386547056642,25769805053,2019,2
8,A-1992188,2019-02-19,3,0.498,At Washington Rd/Exit 1 - Accident.,43.0,19.6,0.27,KATL,1649267441664,25769805053,2019,2
9,A-1992193,2019-02-19,2,0.373,At Moores Mill Rd/Exit 254 - Accident.,39.2,9.2,None,KMGE,1649267441664,25769805053,2019,2


### Example queries

1. **Get the number of accidents in May 2016.**

In [20]:
df = pd.read_sql("SELECT COUNT(*) FROM udacity.accidents WHERE year='2016' AND month='5';", conn)
df

,_col0
0,10489


2. **Get all accidents in Daython, OH**

In [25]:
sql = """
    SELECT city_name, state, description FROM
    udacity.accidents JOIN udacity.cities
    ON udacity.accidents.city_id = udacity.cities.city_id
    WHERE city_name='Dayton' AND state='OH';
"""
df = pd.read_sql(sql, conn)
df

,city_name,state,description
0,Dayton,OH,Accident on Catalpa Dr at Cherry Dr.
1,Dayton,OH,Accident on Gettysburg Ave at Stony Hollow Rd.
2,Dayton,OH,Accident on OH-725 Miamisburg Centerville Rd a...
3,Dayton,OH,Accident on I-75 Northbound at Exits 54A 54B O...
4,Dayton,OH,Accident on OH-725 Miamisburg Centerville Rd n...
...,...,...,...
10446,Dayton,OH,Accident on Gettysburg Ave at Frytown Rd.
10447,Dayton,OH,Accident on OH-725 Miamisburg Centerville Rd b...
10448,Dayton,OH,Accident on 2nd St between Perry St and Red Cr...
10449,Dayton,OH,Accident on 1st St at Keowee St.


3. **Get the number of accidents grouped by state**

In [28]:
sql = """
    SELECT state, COUNT(*) as accident_count FROM
    udacity.accidents JOIN udacity.cities
    ON udacity.accidents.city_id = udacity.cities.city_id
    GROUP BY state
    ORDER BY accident_count DESC;
"""
df = pd.read_sql(sql, conn)
df




,state,accident_count
0,CA,305718
1,TX,216909
2,FL,118603
3,NC,87936
4,MI,39228
5,AZ,36619
6,WA,36404
7,OK,33271
8,GA,33060
9,LA,32884
